In [3]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings import DashScopeEmbeddings

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = '../../../file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf'
docs = PyMuPDFLoader(file_path).load()

In [5]:
docs

[Document(metadata={'source': '../../../file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'file_path': '../../../file/79523710-喀纳斯+环天山丨北疆全景12日(首次入疆推荐).pdf', 'page': 0, 'total_pages': 16, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'WPS 文字', 'producer': 'macOS 版本14.2.1（版号23C71） Quartz PDFContext', 'creationDate': "D:20240808094919+01'49'", 'modDate': "D:20240808094919+01'49'", 'trapped': ''}, page_content='喀纳斯+环天山丨北疆全景12 日(首次入疆推荐)\n活动咨询\n电话咨询\n400-670-6300 [周边游转“1”，国内游转“2”，出境游转“3”，团队定制\n转“4”]\n产品经理\n文武：17858622475；飞翔：17316934053（咨询时间:工作日9:00-18:00）\n企微咨询\n微信在线咨询：游小侠微信：yxkkf6，回复【新疆夏季】入群\n产品经理说\n6 月19 日更新:巴音布鲁克景区已恢复开放\n活动亮点\n经典行程：私家策划大环线，自然风光+璀璨人文：森林、草原、雪山、湖泊、花海、湿地、沙漠、戈\n壁、峡谷、遗址、人文\n适时适地：春夏季的北疆，气候凉爽适宜，草原花开如毯、瓜果成熟飘香、美食众多，是旅行出行之佳选\n独特玩法：安排3 晚景区内住宿，多次日出日落，纯玩不购物，充分游玩时间\n旅途舒适：4 人成行，上限25 人，宽敞舒适大巴，住宿优化升级\n0>\n活动描述\n第1 天全国各地[飞机]或[火车]乌鲁木齐（★全天集合日）\n· 全天\n今天是活动集合日。各位游侠自行前往新疆自治区首府【乌鲁木齐】的集合酒店，报“游侠客+名字”就\n可以办理入住。\n乌鲁木齐位于亚洲的中心，是举世闻名的歌舞之乡、瓜果之乡、黄金玉石之邦。如果时间比较宽裕，可以\n自

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [7]:
len(splits)

16

In [8]:
from langchain.vectorstores import Chroma

persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/data/chroma_vector_db'
collection_name = 'chat-pdf-index'
# 创建
vectorstore = Chroma.from_documents(
    splits,
    qw_embedding,
    collection_name=collection_name,
    persist_directory=persist_directory
)

NameError: name 'splits' is not defined

In [14]:
retriever = vectorstore.as_retriever()
retriever.search_kwargs['k'] = 20

In [15]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(qw_llm_openai, retriever=retriever)

In [16]:
chat_history = []


def ask(question, chat_history):
    response = qa.invoke({"question": question, "chat_history": chat_history})
    chat_history.append((question, response['answer']))
    print(f"Question:\n {question}\n")
    print(f"Answer:\n {response['answer']}\n")

In [17]:
query = '总结一下每日的行程'

In [18]:
ask(query, chat_history)

Question:
 总结一下每日的行程

Answer:
 每日的行程如下:

1. 第1天:
- 各地往返乌鲁木齐的费用(请在报名缴费成功后再预订大交通)
- 保险
- 食宿
- 餐食
- 地接费
- 签证

2. 第2天:
- 乌鲁木齐-吐鲁番:约80KM
- 乌鲁木齐-和静:约340KM

3. 第3天:
- 和静-吐峪沟:约80KM
- 吐峪沟-火焰山:约100KM
- 和静-和静县网评3-4钻商务酒店标间

4. 第4天:
- 和静-察汗努尔达坂:约300KM
- 巴音布鲁克:约400KM

5. 第5天:
- 和静-巴音布鲁克:约400KM
- 巴音布鲁克-喀拉峻草原:约200KM

6. 第6天:
- 喀拉峻草原:约200KM
- 喀拉峻草原-伊宁:约140KM

7. 第7天:
- 伊宁-赛里木湖:约280KM
- 伊宁-精河/奎屯:约300KM

8. 第8天:
- 精河/奎屯-阿勒泰:约320KM

9. 第9天:
- 阿勒泰-乌鲁木齐:约400KM

10. 第10天:
- 乌鲁木齐-禾木:约200KM
- 乌鲁木齐-喀纳斯:约200KM

11. 第11天:
- 喀纳斯-五彩滩:约160KM
- 五彩滩-北屯市:约110KM

12. 第12天:
- 北屯市-乌鲁木齐:约440KM

13. 第13天:
- 乌鲁木齐-北屯市:约440KM

14. 第14天:
- 北屯市-乌鲁木齐:约440KM

15. 第15天:
- 乌鲁木齐-乌鲁木齐:约440KM

16. 第16天:
- 乌鲁木齐-乌鲁木齐:约440KM

以上就是每日的行程安排。


In [20]:
chat_history = []

In [24]:
query = '总结一下每日的行程，用中文回答'
ask(query, chat_history)

Question:
 总结一下每日的行程，用中文回答

Answer:
 The daily itinerary of this trip includes:

Day 1: Fly or train from Beijing to Urumqi, meet at Urumqi and check into a hotel for the night.
Day 2: Visit the Xinjiang Museum in Urumqi and explore the two-storey Bazaar in the city center. Enjoy dinner at a local restaurant.
Day 3: Drive to Kucha (吐鲁番), visit the坎儿井, learn about ancient irrigation techniques used by the locals. Have dinner at a local restaurant.
Day 4: Drive to Turpan, visit the Karez irrigation system, experience the life of the local people, have lunch at a local restaurant, and drive to and stay at a hotel in Turpan.
Day 5: Drive to Hotan, visit the Kunlun Mountains, enjoy a traditional lunch at a local restaurant, and continue driving to and stay at a hotel in Hotan.
Day 6: Drive to Ulan Quxi (阿勒泰), visit the Ular Valley and the Tianchi Lake, enjoy a picnic lunch with the local population, and continue driving to and stay at a hotel in Ulan Quxi.
Day 7: Drive to Urumqi, visit the 

In [21]:
chat_history

[]

In [15]:
query = '判断节点是什么？如何设置？'
ask(query, chat_history)

Question:
 判断节点是什么？如何设置？

Answer:
 判断节点在流程中用于通过提取信息的条件判断串联各节点，它可以根据提取的变量来实现机器人的自动回复。设置时，你可以对一个实体值或一个变量值进行判断。以下是设置的关键点：

- 需要修改的设置项可能包括判断条件，这可以是对某个特定实体或变量的检查。
- 你可以根据业务逻辑设置条件，例如，如果用户提供的信息满足特定条件，机器人将执行相应的回复或操作。

由于没有具体的设置界面或详细步骤，我无法提供详细的设置过程。通常，你可能需要在后台编辑器中选择“添加判断节点”，然后配置判断条件、结果分支（当条件为真和为假时的回复或动作）以及可能涉及的变量。如果你有更具体的问题或者需要了解某个平台的详细步骤，可以告诉我，我会尽力帮助解答。


==================================================================================================================

In [24]:
# 新方式
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
# from langchain.chains import (create_history_aware_retriever, create_retrieval_chain, )
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Contextualize question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    qw_llm_openai, retriever, contextualize_q_prompt
)

# Answer question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# Below we use create_stuff_documents_chain to feed all retrieved context
# into the LLM. Note that we can also use StuffDocumentsChain and other
# instances of BaseCombineDocumentsChain.
question_answer_chain = create_stuff_documents_chain(qw_llm_openai, qa_prompt)
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [ ]:
# Usage:
chat_history = []  # Collect chat history here (a sequence of messages)
rag_chain.invoke({"input": query, "chat_history": chat_history})

In [25]:
chat_history = []


def ask_new(question, chat_history):
    response = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.append((question, response['answer']))
    print(f"Question:\n {question}\n")
    print(f"Answer:\n {response['answer']}\n")

In [18]:
query = '总结一下这个文档'
ask_new(query, chat_history)

Question:
 总结一下这个文档

Answer:
 这段文档介绍了如何创建新流程和理解不同类型的节点。"新建流程"涉及选择模板或创建空白模板，并输入流程名称。"查询节点"用于机器人调用业务系统进行查询，基于用户信息，返回结果给用户或作为其他节点的判断条件，关键设置包括请求地址、请求方式和输入变量。"收集节点"则用来收集客户信息，可以对接到工单、客户中心或第三方系统，支持设置引导语、是否允许附件上传及自定义表单字段。


In [26]:
query = '回复节点是什么呢'
ask_new(query, chat_history)

Question:
 回复节点是什么呢

Answer:
 回复节点是流程中的一个环节，它用来在特定节点位置提供定制化的回复给用户。你可以选择系统应答样式、系统反问样式或自定义样式来设计回复。此外，可以通过添加富文本、按钮选项来构建回复内容，并能设置点击按钮后跳转到下一节点或外部链接。还可以提取变量，以便在用户选择后用于调用业务接口。
